In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.backend import ctc_batch_cost
from sklearn import preprocessing 

sys.path.append('../')
from utils.image_to_tensor import convert_image_to_tensor
from utils.pymongo_get_database import get_database


In [ ]:
dbname = get_database()
quran_words_collection = dbname["quran_words"]
quran_words = quran_words_collection.find()

## Data Loader


In [ ]:
class ImageDataLoader:
    def __init__(self, data_path, fonts, db_name, collection_name):
        self.data_path = data_path
        self.fonts = fonts
        self.db_name = db_name
        self.collection_name = collection_name
        self.db = self._get_database()
        self.images_tensors = []
        self.label_encoder = preprocessing.LabelEncoder()  # Initialize label encoder

    def _get_database(self):
        """Retrieve the MongoDB database."""
        return get_database()

    def _load_images(self):
        """Load images from the specified fonts and convert to tensors."""
        image_number = 0
        for font in self.fonts:
            fonts_dir_paths = os.path.join(self.data_path, font)
            images = os.listdir(fonts_dir_paths)
            for image in images:
                image_path = os.path.join(fonts_dir_paths, image)
                self.images_tensors.append(convert_image_to_tensor(image_path))
                image_number += 1
        print(image_number)

    def get_images_tensors(self):
        """Load images and convert them to tensors, then return them as a NumPy array."""
        self._load_images()
        return np.array(self.images_tensors)

    def fetch_quran_words(self):
        """Fetch Quran words from the database and return them."""
        collection = self.db[self.collection_name]
        return list(collection.find())

    def encode_quran_words(self):
        """Fetch Quran words from the database and encode them using one-hot encoding."""
        quran_words = self.fetch_quran_words()
        words = [word_data["word"] for word_data in quran_words]  # Extract words from the data

        # Create a mapping from word to index
        unique_words = list(set(words))
        word_to_index = {word: idx for idx, word in enumerate(unique_words)}
        num_classes = len(unique_words)  # This will be the number of unique words

        # Convert words to indices
        transformed_words = np.array([word_to_index[word] for word in words])

        # Convert indices to one-hot encoding
        one_hot_labels = tf.keras.utils.to_categorical(transformed_words, num_classes=num_classes)

        # Define the required shape (e.g., (146008, num_classes))
        required_shape = (146008, num_classes)

        # Repeat and truncate one-hot encoded labels to match the required shape
        num_repeats = int(np.ceil(np.prod(required_shape) / one_hot_labels.size))
        repeated_labels = np.tile(one_hot_labels, (num_repeats, 1))[:required_shape[0]]

        return repeated_labels

In [ ]:
data_path = '../../../static/processed/v2/images/'
fonts = ["eljaza", "hijazi", "naskh", "ruqah", "thuluth", "muhammadi", "almaghribi", "uthmanic"]
dbname = "quran_database"
collection_name = "quran_words"

In [ ]:
# Initialzing the data loader
data_loader = ImageDataLoader(data_path, fonts, dbname, collection_name)


In [ ]:
# Transform the images to tensors
x = data_loader.get_images_tensors()
print("Encoded images:", x[0])
print("Encoded images shape:", x.shape)

# Encode Quran words and print the shape of the encoded labels
y = data_loader.encode_quran_words()
print("Encoded words:", y)
print("Encoded words shape:", y.shape)

In [ ]:
print("Encoded images:", x[0].shape)


In [ ]:
# # Neural Network

# model = models.Sequential()

# # CNN Layers 

# # Layer 1: Conv2D + MaxPooling

# model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=(192, 64, 1))) 
# model.add(layers.MaxPooling2D((2, 2)))  # Output: (32, 8, 32)

# # Layer 2: Conv2D + MaxPooling
# model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))  # Output: (32, 8, 64)
# model.add(layers.MaxPooling2D((2, 2)))  # Output: (16, 4, 64)

# # Layer 3: Conv2D + MaxPooling
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))  # Output: (16, 4, 128)
# model.add(layers.MaxPooling2D((1, 2)))  # Output: (16, 2, 128)

# # Layer 4: Conv2D + MaxPooling
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))  # Output: (16, 2, 128)
# model.add(layers.MaxPooling2D((1, 2)))  # Output: (16, 1, 128)

# # Layer 5: Conv2D + MaxPooling
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))  # Output: (16, 1, 256)
# model.add(layers.MaxPooling2D((1, 1), padding='valid'))  # Output: (16, 1, 256)

# # Reshape the output to fit the LSTM input requirements
# model.add(layers.Reshape((192, 256)))
# # BDLTSM Layers
# model.add(Bidirectional(
#     tf.keras.layers.LSTM(265, return_sequences=True)
# ))
# model.add(Bidirectional(
#     tf.keras.layers.LSTM(265, return_sequences=True)
# ))

# model.summary()


In [ ]:
# model.compile(optimizer=RMSprop())
# model.fit(x, y, batch_size=32, epochs=10)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# Define the model
model = models.Sequential()

# CNN Layers
# Layer 1: Conv2D + MaxPooling
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=(192, 64, 1)))
model.add(layers.MaxPooling2D((2, 2)))  # Output: (96, 32, 32)

# Layer 2: Conv2D + MaxPooling
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))  # Output: (48, 16, 64)

# Layer 3: Conv2D + MaxPooling
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((1, 2)))  # Output: (48, 8, 128)

# Layer 4: Conv2D + MaxPooling
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((1, 2)))  # Output: (48, 4, 128)

# Layer 5: Conv2D + MaxPooling
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((1, 1), padding='valid'))  # Output: (48, 4, 256)

# Flatten the output
model.add(layers.Flatten())

# Dense Layer for Classification
model.add(layers.Dense(18251, activation='softmax'))  # Replace num_classes with the number of classes

# Model summary
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
model.fit(x, y, batch_size=32, epochs=10)  